source: https://www.kaggle.com/ejunichi/m5-simple-fe from https://www.kaggle.com/ejunichi/m5-three-shades-of-dark-darker-magic

In [1]:
import sys
import os
import pathlib
import gc
import pandas as pd
pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_rows', 500)
import numpy as np
import math
import random
import pickle
import time
import psutil
import warnings

# custom import
from sklearn.preprocessing import LabelEncoder
from multiprocessing import Pool        # Multiprocess Runs

# warnings.filterwarnings('ignore')

# constant variables for helper functions

In [2]:
N_CORES = psutil.cpu_count()     # Available CPU cores
print(f"N_CORES: {N_CORES}")

N_CORES: 16


# function nicely diplaying a head of Pandas DataFrame

In [3]:
import IPython

def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df)

# function fixing random seeds

In [4]:
def seed_everything(seed=0):
    """Sets seed to make all processes deterministic     # type: int
    
    """
    random.seed(seed)
    np.random.seed(seed)

SEED = 42
seed_everything(SEED)    

# function processing df in multiprocess

In [5]:
def run_df_in_multiprocess(func, t_split):
    """Process ds in Multiprocess
    
    """
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

# other helper functions

In [6]:
def get_memory_usage():
    """メモリ使用量を確認するためのシンプルな「メモリプロファイラ」
    
    """
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)


def merge_by_concat(df1, df2, merge_on):
    """dtypesを失わないための連結による結合
    
    """
    
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

#  constant variables for data import

In [7]:
# _DATA_DIR = os.path.sep.join(["data", "M5_Three_shades_of_Dark_Darker_magic", "sample"])
_DATA_DIR = os.path.sep.join(["data", "M5_Three_shades_of_Dark_Darker_magic"])

_BASE_GRID_FOR_DARKER_MAGIC_EVALUATION = "base_grid_for_darker_magic_evaluation.pkl"

# function importing data

In [8]:
def reduce_mem_usage(df, verbose=True):
    """
    reduce the memory usage of the given dataframe.
    https://qiita.com/hiroyuki_kageyama/items/02865616811022f79754
    
    Args:
        df: Dataframe
        verbose: 
        
    Returns:
        df, whose memory usage is reduced.

    Raises:
        None
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns毎に処理
        col_type = df[col].dtypes
        if col_type in numerics: #numericsのデータ型の範囲内のときに処理を実行. データの最大最小値を元にデータ型を効率的なものに変更
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def read_data(directory, file_name):
    print('Reading files...')
    df = pd.read_csv(os.path.sep.join([str(directory), _DATA_DIR, file_name]))
    df = reduce_mem_usage(df)
    print('{} has {} rows and {} columns'.format(file_name, df.shape[0], df.shape[1]))
    
    return df

# read pickle

In [10]:
parent_dir = pathlib.Path(os.path.abspath(os.curdir)).parent.parent
import_file_name = _BASE_GRID_FOR_DARKER_MAGIC_EVALUATION

grid_df = pd.read_pickle(os.path.sep.join([str(parent_dir), _DATA_DIR, import_file_name]))
print(f"grid_df: {grid_df}")

grid_df:                                      id        item_id    dept_id   cat_id  \
0         HOBBIES_1_008_CA_1_evaluation  HOBBIES_1_008  HOBBIES_1  HOBBIES   
1         HOBBIES_1_009_CA_1_evaluation  HOBBIES_1_009  HOBBIES_1  HOBBIES   
2         HOBBIES_1_010_CA_1_evaluation  HOBBIES_1_010  HOBBIES_1  HOBBIES   
3         HOBBIES_1_012_CA_1_evaluation  HOBBIES_1_012  HOBBIES_1  HOBBIES   
4         HOBBIES_1_015_CA_1_evaluation  HOBBIES_1_015  HOBBIES_1  HOBBIES   
...                                 ...            ...        ...      ...   
47735392    FOODS_3_823_WI_3_evaluation    FOODS_3_823    FOODS_3    FOODS   
47735393    FOODS_3_824_WI_3_evaluation    FOODS_3_824    FOODS_3    FOODS   
47735394    FOODS_3_825_WI_3_evaluation    FOODS_3_825    FOODS_3    FOODS   
47735395    FOODS_3_826_WI_3_evaluation    FOODS_3_826    FOODS_3    FOODS   
47735396    FOODS_3_827_WI_3_evaluation    FOODS_3_827    FOODS_3    FOODS   

         store_id state_id       d  sales  release  wm

# constant variables for preprocessing/prediction

In [11]:
# 予測期間とitem数の定義 / number of items, and number of prediction period
_NUM_UNIQUE_ITEM_ID = 30490
_DAYS_FOR_PREDICTION = 28

DAYS_PER_YEAR = 365
_NUM_YEARS_FOR_MELT = 2
_NUM_IMPORT_ROWS_FOR_MELT = DAYS_PER_YEAR * _NUM_YEARS_FOR_MELT * _NUM_UNIQUE_ITEM_ID
print(f"_NUM_IMPORT_ROWS_FOR_MELT: {_NUM_IMPORT_ROWS_FOR_MELT}")

TARGET = 'sales'
MAIN_INDEX = ['id','d']  # We can identify items by these columns

_NUM_IMPORT_ROWS_FOR_MELT: 22257700


# additionally clean grid_df 

In [12]:
# just remove the prefix "d_" from 'd' column
grid_df['d'] = grid_df['d'].apply(lambda x: x[2:]).astype(np.int16)

# 'wm_yr_wk'の削除
# テスト値がtrainにないため
del grid_df['wm_yr_wk']

print(grid_df)

                                     id        item_id    dept_id   cat_id  \
0         HOBBIES_1_008_CA_1_evaluation  HOBBIES_1_008  HOBBIES_1  HOBBIES   
1         HOBBIES_1_009_CA_1_evaluation  HOBBIES_1_009  HOBBIES_1  HOBBIES   
2         HOBBIES_1_010_CA_1_evaluation  HOBBIES_1_010  HOBBIES_1  HOBBIES   
3         HOBBIES_1_012_CA_1_evaluation  HOBBIES_1_012  HOBBIES_1  HOBBIES   
4         HOBBIES_1_015_CA_1_evaluation  HOBBIES_1_015  HOBBIES_1  HOBBIES   
...                                 ...            ...        ...      ...   
47735392    FOODS_3_823_WI_3_evaluation    FOODS_3_823    FOODS_3    FOODS   
47735393    FOODS_3_824_WI_3_evaluation    FOODS_3_824    FOODS_3    FOODS   
47735394    FOODS_3_825_WI_3_evaluation    FOODS_3_825    FOODS_3    FOODS   
47735395    FOODS_3_826_WI_3_evaluation    FOODS_3_826    FOODS_3    FOODS   
47735396    FOODS_3_827_WI_3_evaluation    FOODS_3_827    FOODS_3    FOODS   

         store_id state_id     d  sales  release  
0           

# export the cleaned base grid (grid_part_1)

In [13]:
# 今後の使用のため（モデルトレーニング）pickleファイルとして保存できます
_EXPORT_FILE_NAME = "clearned_base_grid_for_darker_magic_evaluation.pkl"
print("data export start")
grid_df.to_pickle(os.path.sep.join([str(parent_dir), _DATA_DIR, _EXPORT_FILE_NAME]))
print('data export finished. Size:', grid_df.shape)

del grid_df
gc.collect()

data export start
data export finished. Size: (47735397, 9)


0